In [1]:
import time
import os
import json
from datetime import datetime

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from collections import defaultdict


In [39]:
def get_news(n_url):
    """ (string) -> list
    Args:
        n_url(string): 네이버 뉴스 url
    Return:
        news_detail(list)
        해당 네이버 뉴스의 정보 (제목, 날짜, 본문, 언론사)가 담긴 list 반환
    >>> get_news('https://www.edaily.co.kr/news/read?newsId=01305446625741368&mediaCodeNo=257&OutLnkChk=Y')
    ['코로나가 아이들에게 가와사키병 일으킨다', '20200430', '기사 본문', '이데일리']    
    """
    news_detail = []

    breq = requests.get(n_url)
    bsoup = BeautifulSoup(breq.content, 'html.parser')

    title = bsoup.select('h3#articleTitle')[0].text  #대괄호는  h3#articleTitle 인 것중 첫번째 그룹만 가져오겠다.
    news_detail.append(title)

    pdate = bsoup.select('.t11')[0].get_text()[:11]
    news_detail.append(pdate)

    _text = bsoup.select('#articleBodyContents')[0].get_text().replace('\n', " ")
    btext = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_detail.append(btext.strip())
  
    news_detail.append(n_url)
    
    pcompany = bsoup.select('#footer address')[0].a.get_text()
    news_detail.append(pcompany)

    return news_detail

In [40]:
def crawler(maxpage, query, product_list, disease_list, s_date, e_date):
    """(integer, string, string, string, string, string) -> (None)
    Args:
        maxpage(int): 크롤링할 최대 페이지 수
        query(str): 네이버 뉴스 링크 검색 쿼리문 (검색 조건에 따른 키워드 조합)
        product_list(str): 검색할 제품 단어
        disease_list(str): 검색할 증상 단어
        s_date(str): 크롤링할 시작 날짜
        e_date(str): 크롤링할 끝 날짜
    Return:
        None
        '기사 제목, 기사 배포 날짜, 기사 본문, 언론사'를 기록한 txt 파일 작성
    >>> crwalwer(10, '자전거, 전동퀵보드', '부상, 골절상, 타박상', '2019.01.01', '2020.01.01')
    '자전거, 전동퀵보드'+'부상, 골절상, 타박상'+'.txt' 파일 작성
    """
    
    # input 날짜 (2019.01.01)에서 '.' 제거
    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1
    #maxpage_t: 입력한 maxpage에 따른 네이버 뉴스 기사 수 (네이버뉴스 검색 1page당 10개의 기사가 있음)
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지
    f = open(RESULT_PATH + product_list + disease_list +".txt", 'w', encoding='utf-8')
    
    while page < maxpage_t:
    
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=0&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        req = requests.get(url)
        cont = req.content
        soup = BeautifulSoup(cont, 'html.parser')

        
        for urls in soup.select("._sp_each_url"):
            try :
                # 네이버뉴스링크가 존재하는 것만 수집
                if urls["href"].startswith("https://news.naver.com"):
                    # get_news()함수를 통해 조건에 맞는 뉴스정보를 news_detail에 저장한 후 파일에 입력
                    news_detail = get_news(urls["href"])
                    f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(product_list, disease_list, news_detail[1], news_detail[4], news_detail[0], news_detail[2], news_detail[3]))  # new style
            except Exception as e:
                print(e)
                continue
        page += 10
    
    
    f.close()

In [41]:
def excel_make(product_list,disease_list):
    """ (string, string) -> (None)
    Args:
        product_list(string) : 검색할 제품 단어
        disease_list(string) : 검색할 증상 단어
    Return:
        None
        crawling.txt -> crawling.xlsx 전환
    >>> excel_make('자전거, 전동퀵보드', '부상, 골절상, 타박상')
    '자전거, 전동퀵보드'+'부상, 골절상, 타박상'+'.csv' 파일 작성 (txt -> csv)
    """
    data = pd.read_csv(RESULT_PATH + product_list + disease_list + ".txt", sep='\t',header=None, error_bad_lines=False,engine='python', encoding='utf-8' )
    # pandas dataframe column명 설정
    data.columns = ['product_list','disease_list','years','company','title','contents','link']
    
    
    xlsx_outputFileName = '%s-%s-%s-%s-%s  %s시 %s분 %s초 result.xlsx' % (str(product_list), str(disease_list), now.year, now.month, now.day, now.hour, now.minute, now.second)
    #엑셀 파일로 저장
    data.to_excel(RESULT_PATH+xlsx_outputFileName, encoding='utf-8')


In [42]:
def main():
    # 경로 설정
    RESULT_PATH = 'C:/Users/goddo/바탕 화면/intern/working/result_data/'
    while True:
        
        # 파일저장을 위한 시간 설정
        now = datetime.now()
        keep = input("0 입력시 수집 종료 \n1 입력시 수집 시작\n: ")
        if (keep == "0"):
            break
        
        # 제품명, 증상명, 쿼리문 초기화
        product_list = list()
        disease_list = list()
        query = ''
        maxpage = input("최대 출력할 페이지수 입력하시오: ")
        
        # 증상 입력 -> 제품 입력 순임
        # 논리구조상 '제품 입력 -> 제품과 관련된 증상 입력'이 순서가 맞으나
        # 네이버뉴스 검색어 설정 순서가 '증상(선택 단어)' -> '제품(필수 단어)' 순으로 나타남
        while True:
            disease = input("관련 증상 입력: \nquit입력시 다음 단계로 진행: ")
            if disease == 'quit':
                break
            disease_list.append(disease)
        
        # 뉴스 쿼리문 특성상 마지막 단어에서는 '특수문자가 빠짐'
        for i, disease in enumerate(disease_list):
            if (i != len(disease_list)-1):
                query += disease+'+%7C+'
            else:
                query +=disease
        
        
        while True:
            product = input("제품 이름(꼭 들어가야 하는 단어) 입력: \nquit 입력시 다음 단계로 진행: ")
            if product == 'quit':
                break
            product_list.append(product)
            
        for i, product in enumerate(product_list):
            query += '+%2B'+product


        # 시작 날짜, 마지막 날짜를 input으로 받되, 기본값은 2019.01.01 ~ 2020.01.01
        s_date = input("시작날짜 입력(Default: 2019.01.01): ") or "2019.01.01"  
        e_date = input("끝  날짜 입력(Default: 2020.01.01): ") or "2020.01.01" 

        
        # 파일저장을 위해 list를 string으로 형변환해준다.
        product_list = str(product_list)
        disease_list = str(disease_list)
        
        # input값에 따라 크롤링 시작
        crawler(maxpage,query,product_list,disease_list, s_date,e_date) #검색된 네이버뉴스의 기사내용을 크롤링합니다. 
        
        # 크롤링결과 만들어진 txt -> csv
        excel_make(product_list,disease_list) #엑셀로 만들기     

In [43]:
if __name__ == "__main__":
    main()

0 입력시 수집 종료 
1 입력시 수집 시작10
최대 출력할 페이지수 입력하시오: 10
관련 증상 입력: 
quit입력시 다음 단계로 진행복통
관련 증상 입력: 
quit입력시 다음 단계로 진행설사
관련 증상 입력: 
quit입력시 다음 단계로 진행quit
복통+%7C+설사
제품 이름(꼭 들어가야 하는 단어) 입력: 
quit 입력시 다음 단계로 진행노로바이러스
제품 이름(꼭 들어가야 하는 단어) 입력: 
quit 입력시 다음 단계로 진행quit
노로바이러스
시작날짜 입력(2019.01.01):
끝날짜 입력(2019.04.28):
 
1
https://search.naver.com/search.naver?where=news&query=복통+%7C+설사+%2B노로바이러스&sort=0&ds=&de=&nso=so%3Ar%2Cp%3Afromto%2Ca%3A&start=1
11
https://search.naver.com/search.naver?where=news&query=복통+%7C+설사+%2B노로바이러스&sort=0&ds=&de=&nso=so%3Ar%2Cp%3Afromto%2Ca%3A&start=11
21
https://search.naver.com/search.naver?where=news&query=복통+%7C+설사+%2B노로바이러스&sort=0&ds=&de=&nso=so%3Ar%2Cp%3Afromto%2Ca%3A&start=21
31
https://search.naver.com/search.naver?where=news&query=복통+%7C+설사+%2B노로바이러스&sort=0&ds=&de=&nso=so%3Ar%2Cp%3Afromto%2Ca%3A&start=31
41
https://search.naver.com/search.naver?where=news&query=복통+%7C+설사+%2B노로바이러스&sort=0&ds=&de=&nso=so%3Ar%2Cp%3Afromto%2Ca%3A&start=41
51
https://search.naver.com/search.

Skipping line 27: '	' expected after '"'
Skipping line 15: Expected 9 fields in line 15, saw 18
Skipping line 22: Expected 9 fields in line 22, saw 22


0 입력시 수집 종료 
1 입력시 수집 시작0
